In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy import spatial, signal
import math
from pylab import rcParams
import os
from datetime import datetime, time

### Data Prep

## Dataprep

In [2]:
def df_formatting(df):
    '''
    this function use to prepare df formatting before using function 'Clear_sky'
    Input : raw dataframe (Datetime columns need to be timestamp)
    Output : right format dataframe [right columns name and clean the imperfect day]
    '''
    if set(['Datetime','Solar_irradiance_Wm2']).issubset(df.columns):
        date_col = 'Datetime'
        I_col = 'Solar_irradiance_Wm2'
    else:
        date_col = input('Enter the column name that contains Datatime info: ')
        I_col = input('Enter the column name that contains Irradiance info: ')
    df = df.rename(columns = {date_col:'Datetime', I_col:'I'})
    df['time'] = [d.time() for d in df['Datetime']]
    df['date'] = [d.date() for d in df['Datetime']]
    df.set_index('Datetime',inplace = True)
    df = df.between_time('06:00', '18:00') 
    df.reset_index(inplace = True)
    df = df.dropna(subset = ['I'])
    df.loc[df['I'] < 0 ,'I']  = 0
    #clean the day that miss some data 
    n_data = df.groupby('date').size().mode()[0] 
    df = df.groupby('date').filter(lambda x : len(x) == n_data)
    n_bug = int(input('[sensor bug checking] Input the threshold of consecutive samples that have the same value: '))
    bug_date = list(df[df.groupby('date')['I'].transform(lambda x : abs(x.diff()).rolling(n_bug,center = True).sum()) < 0.5]['date'].unique())
    print('A number of days that have imputational error = ', len(bug_date))
    df = df.drop(df[df['date'].isin(bug_date)].index)
    return df   

### Smoothing Filter : 
<p> the aim is to fit clear sky days with the high resolution (more samples) so we use raw sampling but </p>
<p> we don't want high variance data so we smoothen them 

In [3]:
from scipy.signal import butter, freqz, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

### Visualize The Result : 
to visualize the specific date

In [4]:
def visual_date(list_date, df, col_name = 'I'):
    '''
    this function use to visualize time series of irradiance in selected day 
    Input :df = right format dataframe (right columns name and clean the imperfect day) 
    '''
    df = df.groupby('date')[col_name].apply(lambda x : list(x.dropna()))
    print('total {} days'.format(len(list_date)))
    rcParams['figure.figsize'] = 21, 5
    for i in range(math.ceil(len(list_date)/4)):
        fig, axes = plt.subplots(1, 4)
        for j in range(4):
            if i*4+j < len(list_date):
                axes[j].plot(df[list_date[i*4+j]])
                axes[j].set_title(list_date[i*4+j])

### Clear Sky Detection : 

In [5]:
def clear_sky(df, Q = 0.05, smooth = 'non', visualize = True, save_fig = False):
    ## add smoothing option -- enveloping !!
    ''' 
        select n clear sky days from dataframe of the set of solar irradiance
    Input : 
        1. df -> need column name 'Datetime', 'I' ***
        2. Q -> cosine distance threshold (the less value the less number of clear sky output)
        3. smoothing option (str) (Default = non-smooth) - non, MA, Lowpass 
    Output :
        1. clear sky dataframe with has 1.Datetime, 2.I, 3.I_smooth
    '''
    
    # define smoothing option
    if smooth == 'non':        
        col = 'I'
    elif smooth == 'MA':
        col = 'I_smooth'
        params = int(input('Enter size of window (recommend = 15): '))
        df['I_smooth'] = df.groupby('date')['I'].transform(lambda x : x.rolling(params, center = True).mean())
    elif smooth == 'Lowpass': # need to filter out negative I 
        col = 'I_smooth'
        params = int(input('Enter ratio between cut-off frequency and sampling rate (recommend between 75 to 100 for high sampling rate): '))
        # identify smoothing parameter
        order = 6
        interval = df['Datetime'].diff().median().seconds
        fs = 1/(interval)  # sample rate, Hz
        cutoff = 1/(interval*params)   # desired cutoff frequency of the filter, Hz
        data = np.array(df['I'])
        y = butter_lowpass_filter(data, cutoff, fs, order)
        y = [i if i>= 0 else 0 for i in y] # clean out negative I 
        df['I_smooth'] = y
    else :
        raise ValueError()
        
    # iteratively calculate reference I
    I_ref = df.groupby('time')['I'].apply(lambda x : x.mean()).dropna()
    df_ref = df.copy()
    for i in range(10): 
        feature_df = pd.DataFrame(df_ref.dropna(subset = ['I']).groupby('date')['I']\
                          .apply(lambda x : spatial.distance.cosine(x, I_ref)))
        feature_df.columns = ['cos_dis']
        feature_df.reset_index(inplace = True)
        select_date = feature_df[feature_df['cos_dis'] < feature_df['cos_dis'].quantile(0.50) ]['date'].to_list()
        df_ref = df_ref.loc[df_ref['date'].isin(select_date)]
        if len(select_date) <100:
            break
        I_ref = df_ref.groupby('time')['I'].apply(lambda x : x.mean()).dropna()
    
    # Smoothen I_ref
    cutoff = 1/(interval*50)
    I_ref_lp = butter_lowpass_filter(I_ref, cutoff, fs, order)
    I_ref_lp = [i if i>= 0 else 0 for i in I_ref_lp]
    feature_df = pd.DataFrame(df_ref.dropna(subset = ['I']).groupby('date')['I']\
                          .apply(lambda x : spatial.distance.cosine(x, I_ref_lp)))
    feature_df.columns = ['cos_dis']
    feature_df.reset_index(inplace = True)
    
    # select clearskyday
    select_date = feature_df[feature_df['cos_dis'] < Q ]['date'].to_list() 
    clr_df = df.loc[df['date'].isin(select_date)]
    
    # adjust early morning and late evening error from filter
    clr_df.loc[clr_df['time'] < time(7,0) , 'I_smooth'] = clr_df['I'] 
    clr_df.loc[clr_df['time'] > time(17,0) , 'I_smooth'] = clr_df['I']
    clr_df.reset_index(inplace = True, drop = True)
    

    if visualize:
        rcParams['figure.figsize'] = 21, 5
        print('total {} days'.format(len(select_date)))
        clr_df_raw = clr_df.groupby('date')['I'].apply(lambda x : list(x.dropna()))
        clr_date = clr_df.groupby('date')['Datetime'].apply(lambda x : list(x))
        ymax = clr_df['I'].max()
        if smooth != 'non':
            clr_df_smooth = clr_df.groupby('date')['I_smooth'].apply(lambda x : list(x.dropna()))
        for i in range(math.ceil(len(select_date)/4)):
            fig, axes = plt.subplots(1, 4)
            if smooth != 'non':
                for j in range(4):
                    if i*4+j < len(select_date):
                        axes[j].plot(pd.to_datetime(clr_date[select_date[i*4+j]]),clr_df_raw[select_date[i*4+j]],alpha = 0.4)
                        axes[j].plot(pd.to_datetime(clr_date[select_date[i*4+j]]),clr_df_smooth[select_date[i*4+j]])         
                        axes[j].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M'))
                        axes[j].set_ylim(-50,ymax+100)
                        axes[j].tick_params(axis = 'y',labelsize = 13.5)
                        axes[j].tick_params(axis = 'x',labelsize = 12)
                        axes[j].set_title(select_date[i*4+j], fontsize = 15)
            else:
                for j in range(4):
                    if i*4+j < len(select_date):
                        axes[j].plot(pd.to_datetime(clr_date[select_date[i*4+j]]),clr_df_raw[select_date[i*4+j]])
                        axes[j].xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%H:%M'))
                        axes[j].set_ylim(-50,ymax+100)
                        axes[j].set_title(select_date[i*4+j], fontsize = 15)  
            if save_fig:
                if i == 0 :
                    name = input('Enter name of the power plant: ')
                plt.savefig(name+'_{}.eps'.format(i+1), format = 'eps')
    return clr_df.loc[:,['Datetime','I','I_smooth']]